NNFL ASSIGNMENT

In [ ]:
%tensorflow_version 1.x


In [ ]:
pip install -r lib/Mask_RCNN/requirements.txt

In [ ]:
!pip uninstall tensorflow 
!pip install tensorflow==1.14

!pip uninstall keras 
!pip install keras==2.2.4

In [ ]:
!pip install Keras-Applications

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%load_ext line_profiler

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
sess_config = tf.compat.v1.ConfigProto()

import sys
import os

COCO_DATA = 'data/coco/'
MASK_RCNN_MODEL_PATH = 'lib/Mask_RCNN/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
    
from lib.Mask_RCNN.samples.coco import coco
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
from lib import utils as siamese_utils
from lib import model as siamese_model
from lib import config as siamese_config
   
import time
import datetime
import random
import numpy as np
import skimage.io
import imgaug
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
# train_classes = coco_nopascal_classes
train_classes = np.array(range(1,81))

# Load COCO/val dataset
coco_val = siamese_utils.IndexedCocoDataset()
coco_object = coco_val.load_coco(COCO_DATA, "val", year="2017", return_coco=True)
coco_val.prepare()
coco_val.build_indices()
coco_val.ACTIVE_CLASSES = train_classes

In [ ]:
class SmallEvalConfig(siamese_config.Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1
    NAME = 'coco'
    EXPERIMENT = 'evaluation'
    CHECKPOINT_DIR = 'checkpoints/'
    NUM_TARGETS = 1
    
class LargeEvalConfig(siamese_config.Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1
    NAME = 'coco'
    EXPERIMENT = 'evaluation'
    CHECKPOINT_DIR = 'checkpoints/'
    NUM_TARGETS = 1
    
    # Large image sizes
    TARGET_MAX_DIM = 192
    TARGET_MIN_DIM = 150
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024
    # Large model size
    FPN_CLASSIF_FC_LAYERS_SIZE = 1024
    FPN_FEATUREMAPS = 256
    # Large number of rois at all stages
    RPN_ANCHOR_STRIDE = 1
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 1000
    TRAIN_ROIS_PER_IMAGE = 200
    DETECTION_MAX_INSTANCES = 100
    MAX_GT_INSTANCES = 100

In [ ]:
# The small model trains on a single GPU and runs much faster.
# The large model is the same we used in our experiments but needs multiple GPUs and more time for training.
model_size = 'small' # or 'large'

if model_size == 'small':
    config = SmallEvalConfig()
elif model_size == 'large':
    config = LargeEvalConfig()
    
config.display()

# Provide training schedule of the model
# When evaluationg intermediate steps the tranining schedule must be provided
train_schedule = OrderedDict()
if model_size == 'small':
    train_schedule[1] = {"learning_rate": config.LEARNING_RATE, "layers": "heads"}
    train_schedule[120] = {"learning_rate": config.LEARNING_RATE, "layers": "4+"}
    train_schedule[160] = {"learning_rate": config.LEARNING_RATE/10, "layers": "all"}
elif model_size == 'large':
    train_schedule[1] = {"learning_rate": config.LEARNING_RATE, "layers": "heads"}
    train_schedule[240] = {"learning_rate": config.LEARNING_RATE, "layers": "all"}
    train_schedule[320] = {"learning_rate": config.LEARNING_RATE/10, "layers": "all"}

# Select checkpoint
if model_size == 'small':
    checkpoint = 'checkpoints/small_siamese_mrcnn_0160.h5'
elif model_size == 'large':
    checkpoint = 'checkpoints/large_siamese_mrcnn_coco_full_0320.h5'

In [ ]:
# Load and evaluate model
# Create model object in inference mode.
model = siamese_model.SiameseMaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_checkpoint(checkpoint, training_schedule=train_schedule)
# Evaluate only active classes
active_class_idx = np.array(coco_val.ACTIVE_CLASSES) - 1


In [ ]:
count = 0
for layer in model.keras_model.layers:
  count += 1
  if str(layer).find('Model') != -1:
    print(count, str(layer))
    break
index = count - 1
model.keras_model.layers[index].summary()

In [ ]:
filters, biases = model.keras_model.layers[index].layers[9].get_weights()

filters_min, filters_max = filters.min(), filters.max()
filters = (filters - filters_min)/(filters_max - filters_min)

_, _, n_channels, n_filters = filters.shape

n_channels = 16 #Can't visualise all filters and channels, hence 16 each
n_filters = 16
ind = 1

plt.rcParams["figure.figsize"] = (30,30)

for i in range(n_filters):
  f = filters[:, :, :, i]
  for j in range(n_channels):
    ax = plt.subplot(n_filters, n_channels, ind)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(f[:, :, j], cmap= 'viridis')
    ind += 1

plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
from keras.models import Model
indexes = [17,27,37,49,59,69,79,91,101,111,121,131,141,153,163,173]
outputs = [model.keras_model.layers[index].layers[i].output for i in indexes]
model_feature_maps = Model(inputs=model.keras_model.layers[index].inputs, outputs=outputs)

In [ ]:
# Select category
category = 1
np.random.seed(7)
image_id = np.random.choice(coco_val.category_image_index[category])   
# Load target
target = siamese_utils.get_one_target(category, coco_val, config)
# Load image
image = coco_val.load_image(image_id)
img = np.expand_dims(image, axis=0)
feature_maps = model_feature_maps.predict(img)
plt.imshow(image)

In [ ]:
feature_maps = model_feature_maps.predict(img)
# plot the output from each block
square = 2
s = f"""\n"""
for fmap in feature_maps:
  # plot 64 maps for first 3 layers
  ix = 1
  for _ in range(square):
    for _ in range(square):
      
      # specify subplot
      ax = plt.subplot(square, square, ix)

      # plot filter channel 
      plt.imshow(fmap[0, :, :, ix-1], cmap='viridis')
      ix += 1

  plt.text(0,64,s)
  plt.show()